<a href="https://colab.research.google.com/github/DhyeyR-007/Season-Scene-Neural-Transfer/blob/main/Driver_Code_CycleGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## By DHYEY MANISH RAJANI

In [1]:
#!pip install tensorflow==2.4.1
import tensorflow as tf
print(tf.__version__)

2.9.2


In [20]:
import tensorflow as tf
from random import random
from numpy import load
from numpy import zeros
from numpy import ones
from numpy import asarray
from numpy.random import randint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Concatenate
from matplotlib import pyplot





In [32]:
  ##############################
  ###    Discriminator     ###
  ##############################

def define_discriminator(image_shape):
  init = RandomNormal(stddev=0.2)
  in_image = Input(shape=image_shape)

  ##C64
  d = Conv2D(64, (4,4), strides = (2,2), padding = 'same', kernel_initializer=init) (in_image)
  d = LeakyReLU(alpha=0.2) (d)

  ##C128
  d = Conv2D(128, (4,4), strides = (2,2), padding = 'same', kernel_initializer=init) (d)
  # d = tf.keras.layers.BatchNormalization(axis=[0,1])(axis=-1)(d)
  d = InstanceNormalization(axis=-1)(d)
  d = LeakyReLU(alpha=0.2) (d)

  ##C256
  d = Conv2D(256, (4,4), strides = (2,2), padding = 'same', kernel_initializer=init) (d)
  # d = tf.keras.layers.BatchNormalization(axis=[0,1])(axis=-1)(d)
  d = InstanceNormalization(axis=-1)(d)
  d = LeakyReLU(alpha=0.2) (d)

  ##C512
  d = Conv2D(256, (4,4), strides = (2,2), padding = 'same', kernel_initializer=init) (d)
  # d = tf.keras.layers.BatchNormalization(axis=[0,1])(axis=-1)(d)
  d = InstanceNormalization(axis=-1)(d)
  d = LeakyReLU(alpha=0.2) (d)

  # second last output layer
  d = Conv2D(512, (4,4), padding='same', kernel_initializer=init)(d)
  # d = tf.keras.layers.BatchNormalization(axis=[0,1])(axis=-1)(d)
  d = InstanceNormalization(axis=-1)(d)
  d = LeakyReLU(alpha=0.2)(d)

  # Patch Output
  patch_out = Conv2D(1, (4,4), padding='same', kernel_initializer=init)(d)

  # Model definition
  model = Model(in_image, patch_out)
  model.compile(loss='mse', optimizer=Adam(lr=0.0002, beta_1 = 0.5), loss_weights=[0.5])
  return model

input_shape = (256,256,3)
model_discriminator = define_discriminator(input_shape)
model_discriminator.summary()

Model: "model_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_33 (InputLayer)       [(None, 256, 256, 3)]     0         
                                                                 
 conv2d_224 (Conv2D)         (None, 128, 128, 64)      3136      
                                                                 
 leaky_re_lu_40 (LeakyReLU)  (None, 128, 128, 64)      0         
                                                                 
 conv2d_225 (Conv2D)         (None, 64, 64, 128)       131200    
                                                                 
 instance_normalization_224   (None, 64, 64, 128)      256       
 (InstanceNormalization)                                         
                                                                 
 leaky_re_lu_41 (LeakyReLU)  (None, 64, 64, 128)       0         
                                                          

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [22]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-529elquv
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-529elquv


In [33]:
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization
from keras.utils.vis_utils import plot_model

  ##############################
  ###    Resnet Block     ###
  ##############################
def resnet_block(n_filters, input_layer):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# first convolutional layer
	g = Conv2D(n_filters, (3,3), padding='same', kernel_initializer=init)(input_layer)
	#g = tf.keras.layers.BatchNormalization(axis=[0,1])(axis=-1)(g)

	g = InstanceNormalization(axis=-1)(g)
	g = Activation('relu')(g)
 
	# second convolutional layer
	g = Conv2D(n_filters, (3,3), padding='same', kernel_initializer=init)(g)
	#g = tf.keras.layers.BatchNormalization(axis=[0,1])(axis=-1)(g)
 	
	g = InstanceNormalization(axis=-1)(g)
	# concatenate merge channel-wise with input layer
	g = Concatenate()([g, input_layer])
	return g

  ##############################
  ###    Generator     ###
  ##############################
	
def define_generator(image_shape, n_resnet=9):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# image input
	in_image = Input(shape=image_shape)
	# c7s1-64
	g = Conv2D(64, (7,7), padding='same', kernel_initializer=init)(in_image)
	#g = tf.keras.layers.BatchNormalization(axis=[0,1])(axis=-1)(g)
	g = InstanceNormalization(axis=-1)(g)
	g = Activation('relu')(g)
	# d128
	g = Conv2D(128, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(g)
	#g = tf.keras.layers.BatchNormalization(axis=[0,1])(axis=-1)(g)
	g = InstanceNormalization(axis=-1)(g)
	g = Activation('relu')(g)
	# d256
	g = Conv2D(256, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(g)
	#g = tf.keras.layers.BatchNormalization(axis=[0,1])(axis=-1)(g)
	g = InstanceNormalization(axis=-1)(g)
	g = Activation('relu')(g)
	# R256
	for i in range(n_resnet):
		g = resnet_block(256, g)
	# u128
	g = Conv2DTranspose(128, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(g)
	#g = tf.keras.layers.BatchNormalization(axis=[0,1])(axis=-1)(g)
	g = InstanceNormalization(axis=-1)(g)
	g = Activation('relu')(g)
	# u64
	g = Conv2DTranspose(64, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(g)
	#g = tf.keras.layers.BatchNormalization(axis=[0,1])(axis=-1)(g)
	g = InstanceNormalization(axis=-1)(g)
	g = Activation('relu')(g)
	# c7s1-3
	g = Conv2D(3, (7,7), padding='same', kernel_initializer=init)(g)
	#g = tf.keras.layers.BatchNormalization(axis=[0,1])(axis=-1)(g)
	g = InstanceNormalization(axis=-1)(g)
	out_image = Activation('tanh')(g)
	# define model
	model = Model(in_image, out_image)
	return model
	
img_shape = (256,256,3)
model = define_generator(img_shape)
model.summary()

Model: "model_25"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_34 (InputLayer)          [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_230 (Conv2D)            (None, 256, 256, 64  9472        ['input_34[0][0]']               
                                )                                                                 
                                                                                                  
 instance_normalization_228 (In  (None, 256, 256, 64  128        ['conv2d_230[0][0]']             
 stanceNormalization)           )                                                          

In [24]:
  ############################################################
  ###    Combined Model (Generator+Discriminator)     ###
  ############################################################

def define_composite_model(g_model_1, d_model, g_model_2, image_shape):
  g_model_1.trainable = True
  g_model_2.trainable = False
  d_model.trainable = False

  # generator loss
  input_gen = Input(shape=image_shape)
  gen1_out = g_model_1(input_gen)
  output_d = d_model(gen1_out)   
  
  # identity loss
  input_id = Input(shape=image_shape)
  output_id = g_model_1(input_id)


  # cycle loss forward
  output_f = g_model_2(gen1_out)

  # cycle loss backward
  gen2_out = g_model_2(gen1_out)  ### input_id
  output_b = g_model_1(gen2_out)

  model = Model([input_gen, input_id], [output_d, output_id, output_f, output_b ])
  model.compile(loss=['mse', 'mae', 'mae', 'mae'], optimizer=Adam(lr=0.0002, beta_1 = 0.5), loss_weights=[1, 5, 10, 10])

  return model


		







In [25]:
  ############################################################
  ###    Auxiliary Functions    ###
  ############################################################

def ProvideReal(dataset, n_samples, patch_shape):
 # choose random instances
 ix = randint(0, dataset.shape[0], n_samples)
 # retrieve selected images
 X = dataset[ix]
 # generate 'real' class labels (1)
 y = ones((n_samples, patch_shape, patch_shape, 1))
 return X, y


def ProvideFake(g_model, dataset, patch_shape):
 # generate fake instance
 X = g_model.predict(dataset)
 # create 'fake' class labels (0)
 y = zeros((len(X), patch_shape, patch_shape, 1))
 return X, y


def update_image_pool(pool, images, max_size=50):
		selected = list()
		for image in images:
			if len(pool) < max_size:
			# stock the pool
				pool.append(image)
				selected.append(image)
			elif random() < 0.5:
			# use image, but don't add it to the pool
				selected.append(image)
			else:
			# replace an existing image and use replaced image
				ix = randint(0, len(pool))
				selected.append(pool[ix])
				pool[ix] = image
		return asarray(selected)




In [29]:
model_save_path = '/content/drive/MyDrive/'

In [26]:
  ########################################################################################################################
  ###    Training Combined Models and Discriminators in an alternative/chained fashion as that of conventional GANs     ###
  ########################################################################################################################
def train(Discriminator__Gen_A, Discriminator__Gen_B, Gen_AtoB, Gen_BtoA, combined_AtoB, combined_BtoA, dataset, epochs):

	# no. of epochs and batches
	n_epochs, n_batch, = epochs, 1

	# patch output square shape of the discriminator
	n_patch = Discriminator__Gen_A.output_shape[1]

	
	trainA, trainB = dataset
	poolA, poolB = list(), list()
 
	# batches per epoch
	bat_per_epo = int(len(trainA) / n_batch)
 
	# number of iterations
	n_steps = bat_per_epo * n_epochs


	for i in range(n_steps):
   

	# selecting a batch of real samples
		X_realA, y_realA = ProvideReal(trainA, n_batch, n_patch)
		X_realB, y_realB = ProvideReal(trainB, n_batch, n_patch)
  


	# generating a batch of fake samples
		X_fakeA, y_fakeA = ProvideFake(Gen_BtoA, X_realB, n_patch)
		X_fakeB, y_fakeB = ProvideFake(Gen_AtoB, X_realA, n_patch)
		X_fakeA = update_image_pool(poolA, X_fakeA)
		X_fakeB = update_image_pool(poolB, X_fakeB)
  
	# generator A->B loss
		g_loss1, _, _, _, _ = combined_AtoB.train_on_batch([X_realA, X_realB], [y_realB, X_realB, X_realA, X_realB])
	# discriminator for B check
		dB_loss1 = Discriminator__Gen_B.train_on_batch(X_realB, y_realB)
		dB_loss2 = Discriminator__Gen_B.train_on_batch(X_fakeB, y_fakeB)
  

	# generator B->A loss
		g_loss2, _, _, _, _  = combined_BtoA.train_on_batch([X_realB, X_realA], [y_realA, X_realA, X_realB, X_realA])
	# discriminator for A check
		dA_loss1 = Discriminator__Gen_A.train_on_batch(X_realA, y_realA)
		dA_loss2 = Discriminator__Gen_A.train_on_batch(X_fakeA, y_fakeA)
  

	## Saving and plotting training models
		if (i+1) % (bat_per_epo * 1) == 0:
				combined_AtoB.save(model_save_path + 'combined_AtoB.h5')
				combined_BtoA.save(model_save_path + 'combined_BtoA.h5')
		
				Gen_AtoB.save(model_save_path + 'Gen_AtoB.h5')
				Gen_BtoA.save(model_save_path + 'Gen_BtoA.h5')
		
				Discriminator__Gen_A.save(model_save_path + 'Discriminator__Gen_A.h5')
				Discriminator__Gen_B.save(model_save_path + 'Discriminator__Gen_B.h5')

				X_real_A_test, y_real_A_test = ProvideReal(trainA,n_batch,n_patch)
				### X_real_B_test, y_real_B_test = generateRealSamples(dataset_testB,patchSize,batchSize)
		
				X_fake_B_test,y_fake_B_test = ProvideFake(Gen_AtoB,X_real_A_test,n_patch)
				
				X_reconst_A_test,y_reconst_A_test = ProvideFake(Gen_BtoA,X_fake_B_test,n_patch)
		
			# scaling [-1,1] to [0,1]
				X_real_A_test = (X_real_A_test + 1) / 2.0
				X_fake_B_test = (X_fake_B_test + 1) / 2.0
				X_reconst_A_test = (X_reconst_A_test + 1) / 2.0


				print("Iteration start" )
		
				# plot real images
				fig = pyplot.figure(figsize=(10, 10))
				ax = fig.add_subplot(321)
				ax.axis('off')
				ax.title.set_text('Real')
				ax.imshow(np.squeeze(X_real_A_test))
				
				# plot translated image	    
				ax = fig.add_subplot(323)
				ax.axis('off')
				ax.title.set_text('Generated')
				ax.imshow(np.squeeze(X_fake_B_test))
		
		    # plot reconstructed image	    
				ax = fig.add_subplot(325)
				ax.axis('off')
				ax.title.set_text('Reconstructed')
				ax.imshow(np.squeeze(X_reconst_A_test))
		    
				print("Iteration end" )
		
		# summarize performance
		print('Iteration>%d, dA[%.3f,%.3f] dB[%.3f,%.3f] g[%.3f,%.3f]' % (i+1, dA_loss1,dA_loss2, dB_loss1,dB_loss2, g_loss1,g_loss2))
	  


		

In [ ]:
from os import listdir
from numpy import asarray
from numpy import vstack
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from matplotlib import pyplot as plt
import numpy as np
from sklearn.utils import resample


def load_images(path, size=(256,256)):
	data_list = list()
	# enumerate filenames in directory, assume all are images
	for filename in listdir(path):
		# load and resize the image
		pixels = load_img(path + '/'+ filename , target_size=size)
		# convert to numpy array
		pixels = img_to_array(pixels)
		# store
		data_list.append(pixels)
	return asarray(data_list)


# path to dataset
path = '/content/drive/MyDrive/horse2zebra/horse2zebra'


#  train A
dataA_all = load_images(path + '/trainA')
print('Loaded dataA: ', dataA_all.shape)
dataA = resample(dataA_all, 
                 replace=False,     
                 n_samples=5,    
                 random_state=42) 

#   train B
dataB_all = load_images(path + '/trainB')
print('Loaded dataB: ', dataB_all.shape)
dataB = resample(dataB_all, 
                 replace=False,     
                 n_samples=5,    
                 random_state=42) 

# # plot source images
# n_samples = 3
# for i in range(n_samples):
# 	plt.subplot(2, n_samples, 1 + i)
# 	plt.axis('off')
# 	plt.imshow(dataA[i].astype('uint8'))
# # plot target image
# for i in range(n_samples):
# 	plt.subplot(2, n_samples, 1 + n_samples + i)
# 	plt.axis('off')
# 	plt.imshow(dataB[i].astype('uint8'))
# plt.show()



# load image data
data = [dataA, dataB]
print('Loaded', data[0].shape, data[1].shape)

#Preprocess data to change input range to values between -1 and 1 since  generator uses tanh activation in the output layer which ranges between -1 and 1
def preprocess_data(data):
	# load compressed arrays
	# unpack arrays
	X1, X2 = data[0], data[1]
	# scale from [0,255] to [-1,1]
	X1 = (X1 - 127.5) / 127.5
	X2 = (X2 - 127.5) / 127.5
	return [X1, X2]

dataset = preprocess_data(data)

image_shape = dataset[0].shape[1:]




## gen A -> B
Gen_AtoB =  define_generator(image_shape)

## gen B -> A
Gen_BtoA = define_generator(image_shape)


## discr A -> real/fake
Discriminator__Gen_A = define_discriminator(image_shape)

## discr B -> real/fake
Discriminator__Gen_B = define_discriminator(image_shape)


## composite A -> B -> real/fake
combined_AtoB = define_composite_model(Gen_AtoB, Discriminator__Gen_B, Gen_BtoA, image_shape)

## composite B -> A -> real/fake 
combined_BtoA = define_composite_model(Gen_BtoA, Discriminator__Gen_A, Gen_BtoA, image_shape)






# train models
train(Discriminator__Gen_A, Discriminator__Gen_B, Gen_AtoB, Gen_BtoA, combined_AtoB, combined_BtoA, dataset, 1)








In [ ]:
## for testing the generator on test data and plotting the test data

from sklearn.utils import resample

## load test data from main dataset

dataA_all_t = load_images(path + '/testA')
print('Loaded dataA_t: ', dataA_all.shape)
dataA_t = resample(dataA_all_t, 
                 replace=False,     
                 n_samples=5,    
                 random_state=42) 


dataB_all_t = load_images(path + '/testB')
print('Loaded dataB: ', dataB_all.shape)
dataB_t = resample(dataB_all_t, 
                 replace=False,     
                 n_samples=5,    
                 random_state=42) 




for i in range(0,dataA_t.shape[0]):  
  X = []  
  imVal = dataA_t[i]
  assert imVal.shape == (256, 256, 3)
  imVal = imVal.astype(np.float32)
  imVal = (imVal-127.5) / 127.5    
  X.append(imVal)

  X_real_A_test = np.asarray(X)
  X_fake_B_test,y_fake_B_test = ProvideFake(Gen_AtoB,X_real_A_test,16)
  X_reconst_A_test,y_reconst_A_test = ProvideFake(Gen_BtoA,X_fake_B_test,16)

  X_real_A_test = (X_real_A_test + 1) / 2.0
  X_fake_B_test = (X_fake_B_test + 1) / 2.0
  X_reconst_A_test = (X_reconst_A_test + 1) / 2.0

  print("Test Iteration start" )
  print("Test Iteration end" )

# plot real images
fig = pyplot.figure(figsize=(10, 10))
ax = fig.add_subplot(321)
ax.axis('off')
ax.title.set_text('Test_Real')
ax.imshow(np.squeeze(X_real_A_test))
    
# plot translated image	    
ax = fig.add_subplot(323)
ax.axis('off')
ax.title.set_text('Test_Generated')
ax.imshow(np.squeeze(X_fake_B_test))


# plot reconstructed image	    
ax = fig.add_subplot(325)
ax.axis('off')
ax.title.set_text('Test_Reconstructed')
ax.imshow(np.squeeze(X_reconst_A_test))

